In [1]:
%matplotlib inline
%run ../../path_import.py
from plots import *
from training_reparam import *
from eubo import *
from model_reparam import *

import probtorch
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [2]:
data_path = "../gmm_dataset_c2k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()
_, N, D = Data.shape
K = 3 ## number of clusters
## Model Parameters
MCMC_SIZE = 10
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 64
## Training Parameters
BATCH_SIZE = 20
NUM_EPOCHS = 3000
LEARNING_RATE = 1e-3
CUDA = torch.cuda.is_available()
PATH = 'ag-reparam-symKL'

DEVICE = torch.device('cuda:1')

In [3]:
# initialization
enc_z = Gibbs_z(K, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE)
if CUDA:
    enc_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(enc_eta.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))

In [ ]:
train_cfz_reparam(Eubo_cfz, enc_eta, enc_z, optimizer, Data, K, NUM_EPOCHS, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE, PATH, CUDA, DEVICE)

epoch=0, symKL=5473.117, EUBO=2488.787, ELBO=-3395.169, ESS=1.359 (20s)
epoch=1, symKL=4144.327, EUBO=1924.469, ELBO=-2630.089, ESS=1.379 (20s)
epoch=2, symKL=3490.170, EUBO=1654.296, ELBO=-2242.667, ESS=1.396 (20s)
epoch=3, symKL=3073.815, EUBO=1488.160, ELBO=-1995.745, ESS=1.413 (20s)
epoch=4, symKL=2772.388, EUBO=1342.425, ELBO=-1837.309, ESS=1.444 (20s)


In [ ]:
torch.save(enc_eta.state_dict(), '../weights/enc-eta-%s' + PATH)
torch.save(enc_z.state_dict(), '../weights/enc-z-%s' + PATH)

In [ ]:
BATCH_SIZE_TEST = 50
obs, q_eta, q_z = test(Eubo_idw_init_eta, enc_eta, enc_z, Data, K, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE_TEST, CUDA, DEVICE)